# Análisis tienda ICE

## Descripción del proyecto 
---

Trabajas para la tienda online Ice que vende videojuegos por todo el mundo. Las reseñas de usuarios y expertos, los géneros, las plataformas (por ejemplo, Xbox o PlayStation) y los datos históricos sobre las ventas de juegos están disponibles en fuentes abiertas. Tienes que identificar patrones que determinen si un juego tiene éxito o no. Esto te permitirá detectar proyectos prometedores y planificar campañas publicitarias.

Delante de ti hay datos que se remontan a 2016. Imaginemos que es diciembre de 2016 y estás planeando una campaña para 2017.


### Objetivo 

El objetivo de este proyecto es identificar los factores que determinan el éxito de los videojuegos en función de diversas características y métricas de ventas en distintas plataformas y regiones.

Las acciones a realizar: 
- Paso 1: Cargar y explorar los datos
- Paso 2: Manipulación de datos
- Paso 3: Análisis de datos
- Paso 4: Pruebas de hipótesis

## Cargar y explorar los datos
---

En esta sección, comenzaremos cargando las bibliotecas necesarias y los datos proporcionados. Luego, realizaremos una exploración inicial del conjunto de datos para comprender su estructura. Esto incluirá:

- Verificar el tamaño del dataset (número de filas y columnas)
- Examinar los tipos de datos en cada columna
- Identificar la presencia de valores faltantes o atípicos que puedan requerir tratamiento antes de proceder con el análisis.

### Cargar ambiente
Importamos las bibliotecas esenciales para el análisis.

In [31]:
## Cargar el ambiente de trabajo
import pandas as pd
import numpy as np
import math as mt
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns

### Cargar dataset
Leemos el archivo de datos proporcionad por analizar. 

In [32]:
# cargar los datos
df_games = pd.read_csv("../data/games.csv")

### Exploración del dataset
En esta etapa, realizaremos un análisis preliminar del dataset para comprender su estructura y calidad. 

In [33]:
## Visualización del df
df_games.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [34]:
## Información general
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [35]:
# Valores duplicados exoplícitos
print(df_games.duplicated().sum())

## Identificación de los valores duplicados
df_games[df_games.duplicated()]

0


,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating


In [36]:
# Valores duplicados implícitos
## Número de juegos repetidos
df_games.duplicated(subset = "Name").sum()

games_duplicated = df_games[df_games.duplicated(subset = "Name")].sort_values("Name")

print("Número de juegos repetidos:", len(games_duplicated))
print("Número de juegos únicos repetidos:", games_duplicated["Name"].nunique())

Número de juegos repetidos: 5155
Número de juegos únicos repetidos: 2805


In [37]:
## Valores faltantes
df_games.isna().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
NA_sales              0
EU_sales              0
JP_sales              0
Other_sales           0
Critic_Score       8578
User_Score         6701
Rating             6766
dtype: int64

In [38]:
## Estadística descriptiva para columnas numéricas
round(df_games.describe(),2)

,Year_of_Release,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score
count,16446.00,16715.00,16715.00,16715.00,16715.00,8137.00
mean,2006.48,0.26,0.15,0.08,0.05,68.97
std,5.88,0.81,0.50,0.31,0.19,13.94
min,1980.00,0.00,0.00,0.00,0.00,13.00
25%,2003.00,0.00,0.00,0.00,0.00,60.00
50%,2007.00,0.08,0.02,0.00,0.01,71.00
75%,2010.00,0.24,0.11,0.04,0.03,79.00
max,2016.00,41.36,28.96,10.22,10.57,98.00


In [39]:
## Estadísticas descriptivas para columnas categóricas
df_games.describe(include="object")

,Name,Platform,Genre,User_Score,Rating
count,16713,16715,16713,10014,9949
unique,11559,31,12,96,8
top,Need for Speed: Most Wanted,PS2,Action,tbd,E
freq,12,2161,3369,2424,3990


El DataFrame de Ice contiene 16,715 observaciones y 11 variables, proporciona datos sobre el lanzamiento de videojuegos y las ganancia en 3 principales mercados: Norteamérica, Europa y Japón. 

Durante la exploración inicial, identificamos los siguientes problemas: 

- Los nombres de las columnas se encuentran en *snake case* con mayúsculas.
- La columna `Year_of_Release` se encuentra en formato float. 
- La columna `User_Score` se encuentra como objeto. 

En general, no encontramos datos duplicados explícitos. Sin embargo, al analizar la variable `Name` para duplicados implícitos, encontramos 5,155 casos en los que un mismo videojuego se distribuye en distintas plataformas. Esto sugiere que los datos no están en formato tidy, ya que cada fila no representa una observación única en sí misma. 

Respecto a los valores faltantes: 
- Las variables `Name`, `Genre` y `Year_of_Release` presentan ausencias en escasos registros, representando menos del 5% de los datos. 
- Las variables como `Critic_Score`, `User_Score` y `Rating` muestran entre un 40% y 55% de valores faltantes.  

De manera preliminar, notamos que el juego con mayor distribución entre plataformas el Need for Spped: Most Wantes; la plataforma con un mayor número de videojuegos es la PS2; el género más representativo es de Acción. En cuanto a la clasificación, la más frecuente es la E, lo que indica una preferencia del mercado hacia juegos para toda la familia. 

El mercado estadounidense tiene en promedio el mayor número de ventas. En este mercado se registó el valor máximo de ventas. 

In [40]:
df_games

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16711,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16712,Haitaka no Psychedelica,PSV,2016.0,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16713,Spirits & Spells,GBA,2003.0,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


## Manipulación de datos
---

En esta etapa, abordamos los errores identificados en el DataFrame, lo que nos permite preparar los datos para un análisis más adecuado.

Los errores observados fueron los siguientes: 
- Los nombres de las columnas se encuentran en *snake case* con mayúsculas.
- La columna `Year_of_Release` se encuentra en formato float. 
- La columna `User_Score` se encuentra como objeto. 
- Las variables `Name`, `Genre` y `Year_of_Release` presentan ausencias en escasos registros, representando menos del 5% de los datos. 
- Las variables como `Critic_Score`, `User_Score` y `Rating` muestran entre un 40% y 55% de valores faltantes.  

1. Corregimos el formato *snake case* con mayúsculas a minúsculas.

In [43]:
df_games.columns = df_games.columns.str.lower()

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')


2. Para la variables `year_of_release` hicimos la conversión de tipo float a tipo fecha. 